<a href="https://colab.research.google.com/github/ClarisseAlvarenga/CodeNation-ProjetoPratico/blob/master/Codenation_Projeto_Pr%C3%A1tico_modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROJETO PRÁTICO CODENATION ACELERADEV DS

## Escopo do Projeto

**Objetivo**

O objetivo deste produto é fornecer um serviço automatizado que recomenda leads para um usuário dado sua atual lista de clientes (Portfólio).

---



**Contextualização**


Algumas empresas gostariam de saber quem são as demais empresas em um determinado mercado (população) que tem maior probabilidade se tornarem seus próximos clientes. Ou seja, a sua solução deve encontrar no mercado quem são os leads mais aderentes dado as características dos clientes presentes no portfólio do usuário.

Além disso, sua solução deve ser agnóstica ao usuário. Qualquer usuário com uma lista de clientes que queira explorar esse mercado pode extrair valor do serviço.

---



**Requisitos de Negócio**

Para o desafio, deverão ser consideradas as seguintes bases:

**Mercado:** 

Base com informações sobre as empresas do Mercado a ser considerado. Portfolio 1: Ids dos clientes da empresa 1 Portfolio 2: Ids dos clientes da empresa 2 Portfolio 3: Ids dos clientes da empresa 3

Obs: todas as empresas(ids) dos portfolios estão contidos no Mercado(base de população).

**Bases de Estudo**

Link para download das bases Mercado, Portfolio 1, Portfolio 2 e Portfolio 3 respectivamente:

https://codenation-challenges.s3-us-west-1.amazonaws.com/ml-leads/estaticos_market.csv.zip

https://codenation-challenges.s3-us-west-1.amazonaws.com/ml-leads/estaticos_portfolio1.csv

https://codenation-challenges.s3-us-west-1.amazonaws.com/ml-leads/estaticos_portfolio2.csv

https://codenation-challenges.s3-us-west-1.amazonaws.com/ml-leads/estaticos_portfolio3.csv

As bases de portfólio poderão ser utilizadas para testar a aderência da solução. Além disso, se a equipe desejar, poderá simular portfólios por meio de amostragens no mercado.

**Dicionário de variáveis**

https://s3-us-west-1.amazonaws.com/codenation-challenges/ml-leads/features_dictionary.pdf


---


**Requisitos técnicos obrigatórios**


Utilizar técnicas de data science e machine learning para desenvolver o projeto;
Apresentar o desenvolvimento e outputs do modelo em um Jupyter Notebook ou outra tecnologia de apresentação de Output de modelos de Machine Learning;
A análise deve considerar os seguintes pontos: análise exploratória dos dados, tratamento dos dados, avaliação de algoritmos, treinamento do modelo, avaliação de performance do modelo e visualização dos resultados;
Para a apresentação do projeto, o tempo entre o treinamento do modelo e o output deve ser menor que 20 min.

https://docs.google.com/document/d/1MROnrFg10M_9JWmnFAEpcFmLGoAIPNTDJKltSOUOx8Q/edit

---

**Link para a anális exploratória**


https://colab.research.google.com/drive/12Hi-GeEitn7gOWMCFVJl8EbvXSmaBaj0?usp=sharing

# BAIXANDO OS ARQUIVOS ORIGINAIS

In [2]:
import pandas as pd
import sklearn
import zipfile
# import matplotlib.pyplot as plt
# import seaborn as sns
from zipfile import ZipFile
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder


### Abrindo os arquivos

# PRE-PROCESSAMENTO DE DADOS

In [3]:
def ajusta_arquivo_principal(mercado):
    ## CRIANDO UM DF A PARTIR DAS FEATURES SELECIONADAS DURANTE A ANÁLISE EXPLORATÓRIA
    features = ['id', 'fl_matriz',
                'de_natureza_juridica',
                'sg_uf',
                'setor',
                'idade_emp_cat',
                'fl_email',
                'fl_telefone',
                'fl_rm',
                'nm_divisao',
                'nm_segmento',
                'fl_veiculo',
                'fl_optante_simples',
                'fl_optante_simei',
                'de_saude_tributaria',
                'de_nivel_atividade',
                'nm_meso_regiao',
                'nm_micro_regiao',
                'fl_passivel_iss',
                'qt_socios',
                'idade_media_socios',
                'de_faixa_faturamento_estimado',
                'de_faixa_faturamento_estimado_grupo',
                'qt_filiais']
    dados = mercado[features].copy()
    # transformar o id em index
    dados.set_index('id', inplace=True)
    # ALTERANDO OS DADOS BOOLEANOS PARA BINÁRIOS
    dados = dados.replace({True: 1, False: 0})

    #TRANSFORMAÇÃO DE DADOS NUMÉRICOS EM CATEGÓRICOS
    # Criar classificação se contém filial ou não. Deletar "qt_filiais"
    dados['contem_filial'] = [1 if n > 0 else 0 for n in dados['qt_filiais']]
    del dados['qt_filiais']
    # transformando a coluna qt_socios em categórica
    dados.qt_socios.value_counts(normalize=True)
    pd.cut(dados.qt_socios, bins=[-1, 0, 1, 10, 50, 100, 246], include_lowest=True,
           labels=['s/info', 'único dono', 'até 10', 'entre 10 e 50', 'entre 50 e 100', 'mais de 100'])

    dados['socios_cat'] = pd.cut(dados.qt_socios, bins=[-1, 0, 1, 10, 50, 100, 246], include_lowest=True,
                                 labels=['s/info', 'único dono', 'até 10', 'entre 10 e 50', 'entre 50 e 100',
                                         'mais de 100'])

    dados.drop(['qt_socios'], axis=1, inplace=True)
    # categorizando a idade média dos sócios
    pd.cut(dados.idade_media_socios, bins=[-3, 0, 30, 50, 70, 127],
           labels=['s/info', 'até 30 anos', 'entre 30 e 50', 'entre 50 e 70', 'mais de 70 anos'], include_lowest=True)

    dados['idade_media_socios_cat'] = pd.cut(dados.idade_media_socios, bins=[-2, 0, 30, 50, 70, 127],
                                             labels=['s/info', 'até 30 anos', 'entre 30 e 50', 'entre 50 e 70',
                                                     'mais de 70 anos'])
    # excluindo a variável numérica categorizada
    dados.drop(['idade_media_socios'], axis=1, inplace=True)
    # TRATANDO NULOS
    dados.fillna('s/info', inplace=True)
    #TRANSFORMANDO TODAS AS COLUNAS EM CATEGÓRICAS
    # usando um laço para mudar o tipo da variável para string
    for col in dados.columns:
        dados[col] = dados[col].astype(str)

    return dados


In [4]:
def preprocessa_dados(dados):
    # instanciando o LabelEncoder
    le = LabelEncoder()
    #criando um novo arquivo para lidar com o preprocessamento
    X = dados.copy()
    # Categorical confirmando as variáveis categóricas
    categorical_feature_mask = X.dtypes == object
    # filtrar colunas categóricas e salvá-las numa lista
    categorical_cols = X.columns[categorical_feature_mask].tolist()
    # aplicando o LabelEncoder nas colunas categóricas - POR COLUNA usando uma função lambda
    X[categorical_cols] = X[categorical_cols].apply(lambda col: le.fit_transform(col))
    return X

In [5]:
def cria_modelo():
    # cria o modelo
    nn = NearestNeighbors()
    return nn

In [6]:
def ajusta_portfolio(lista_clientes, base):
    portfolio = lista_clientes[['id']]
    portfolio = portfolio.merge(base, on='id')
    portfolio.set_index('id', inplace=True)
    return portfolio

In [20]:
# criando uma função de retorno
def busca_recomendacoes(portfolio, modelo, X, mercado):
    #ajustando os modelos aos dados
    modelo.fit(X)
    # busca as distâncias e os índices dos vizinhos, tendo o portfolio como parâmetro
    distances, indices = modelo.kneighbors(portfolio)
    print(distances)
    print(indices)
    # cria um df legível com as variáveis originais, usando o dados como padrão
    df_cliente = pd.DataFrame([mercado.iloc[indices[row,col]] for row in range(indices.shape[0]) for col in range(indices.shape[1])]).reset_index()
    print(df_cliente.head())
    return df_cliente

## CRIANDO UM DF A PARTIR DAS FEATURES SELECIONADAS DURANTE A ANÁLISE EXPLORATÓRIA

In [11]:
zip_file = ZipFile('/content/estaticos_market.csv.zip')     
dfs = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename))
for text_file in zip_file.infolist() if text_file.filename.endswith('.csv')}
list(zip(dfs))
# criando o DF de mercado
mercado = pd.DataFrame(dfs['estaticos_market.csv'])
dados = ajusta_arquivo_principal(mercado)
lista_clientes = pd.DataFrame(pd.read_csv('/content/estaticos_portfolio1.csv'))
        
        

In [12]:
dados.head()

,fl_matriz,de_natureza_juridica,sg_uf,setor,idade_emp_cat,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_veiculo,fl_optante_simples,fl_optante_simei,de_saude_tributaria,de_nivel_atividade,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,contem_filial,socios_cat,idade_media_socios_cat
id,,,,,,,,,,,,,,,,,,,,,,,
a6984c3ae395090e3bee8ad63c3758b110de096d5d819583a784a113726db849,1,SOCIEDADE EMPRESARIA LIMITADA,RN,CONSTRUÇÃO CIVIL,10 a 15,1,1,SIM,CONSTRUCAO DE EDIFICIOS,CONSTRUCAO,0.0,1.0,0.0,VERDE,ALTA,LESTE POTIGUAR,NATAL,1.0,"DE R$ 1.500.000,01 A R$ 4.800.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00",0,até 10,entre 30 e 50
6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c47fdef50b33f4f56f05,1,EMPRESARIO INDIVIDUAL,PI,SERVIÇO,1 a 5,1,1,SIM,ALIMENTACAO,ALOJAMENTO E ALIMENTACAO,0.0,s/info,s/info,CINZA,BAIXA,CENTRO NORTE PIAUIENSE,TERESINA,1.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",0,único dono,até 30 anos
4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02edc5147bc2a2535b08,1,EMPRESARIO INDIVIDUAL,AM,SERVIÇO,5 a 10,0,1,SIM,TRANSPORTE TERRESTRE,TRANSPORTE ARMAZENAGEM E CORREIO,0.0,1.0,1.0,AMARELO,MEDIA,CENTRO AMAZONENSE,MANAUS,1.0,"ATE R$ 81.000,00","ATE R$ 81.000,00",0,único dono,entre 30 e 50
3348900fe63216a439d2e5238c79ddd46ede454df7b9d8c24ac33eb21d4b21ef,1,EMPRESARIO INDIVIDUAL,AM,SERVIÇO,5 a 10,1,1,SIM,REPARACAO E MANUTENCAO DE EQUIPAMENTOS DE INFO...,OUTRAS ATIVIDADES DE SERVICOS,0.0,1.0,0.0,AMARELO,MEDIA,CENTRO AMAZONENSE,MANAUS,1.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",0,único dono,entre 30 e 50
1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4b102c769f7611ab29f,1,EMPRESARIO INDIVIDUAL,RN,SERVIÇO,1 a 5,1,1,SIM,SERVICOS DE ARQUITETURA E ENGENHARIA TESTES E ...,ATIVIDADES PROFISSIONAIS CIENTIFICAS E TECNICAS,0.0,1.0,0.0,VERDE,ALTA,LESTE POTIGUAR,NATAL,1.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",0,único dono,s/info


In [13]:
X = preprocessa_dados(dados)
X.head()

,fl_matriz,de_natureza_juridica,sg_uf,setor,idade_emp_cat,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_veiculo,fl_optante_simples,fl_optante_simei,de_saude_tributaria,de_nivel_atividade,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,contem_filial,socios_cat,idade_media_socios_cat
id,,,,,,,,,,,,,,,,,,,,,,,
a6984c3ae395090e3bee8ad63c3758b110de096d5d819583a784a113726db849,1,60,4,2,1,1,1,1,32,10,0,1,0,4,0,6,48,1,2,2,0,0,1
6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c47fdef50b33f4f56f05,1,17,3,4,0,1,1,1,3,3,0,2,2,2,1,4,69,1,10,10,0,5,0
4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02edc5147bc2a2535b08,1,17,1,4,3,0,1,1,86,20,0,1,1,0,2,2,43,1,1,1,0,5,1
3348900fe63216a439d2e5238c79ddd46ede454df7b9d8c24ac33eb21d4b21ef,1,17,1,4,3,1,1,1,74,17,0,1,0,0,2,2,43,1,10,10,0,5,1
1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4b102c769f7611ab29f,1,17,4,4,0,1,1,1,77,8,0,1,0,4,0,6,48,1,10,10,0,5,4


In [17]:
modelo = cria_modelo()
base = X.reset_index()
portfolio = ajusta_portfolio(lista_clientes, base)

        

In [18]:
portfolio.head()

,fl_matriz,de_natureza_juridica,sg_uf,setor,idade_emp_cat,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_veiculo,fl_optante_simples,fl_optante_simei,de_saude_tributaria,de_nivel_atividade,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,contem_filial,socios_cat,idade_media_socios_cat
id,,,,,,,,,,,,,,,,,,,,,,,
dabe79bec87c88ae04e869bf6bd321ee5e1893cecf66255be9ea861a9e5841a9,1,17,3,1,4,0,0,0,30,9,0,2,2,2,1,14,61,0,10,10,0,5,1
32e5f4e10932153a7ba869cb0386e7e02d49d2461046b866411556d2af150702,0,17,2,1,3,1,1,0,30,9,0,0,0,4,2,3,44,0,10,7,1,4,4
a95d6f30bba445bd3d6b0c5b36f865b38ec01d1733609095d359649cfedf15d1,1,17,2,1,3,0,0,1,30,9,0,1,1,1,2,8,0,0,1,1,0,5,2
6cb309685cea0b6d2988818792ec2e6fcb2bd02e0afa9e74532d5040fbc933b8,1,17,4,1,5,0,0,0,30,9,0,1,1,4,0,11,70,0,1,1,0,5,1
f72aa7fa6787b0a5a1c88885b6120850df8ee0f71adc251dfe8d321e9e6de7f5,1,60,4,1,2,1,1,1,30,9,0,1,0,4,0,6,48,0,2,8,1,0,1


In [21]:
leads = busca_recomendacoes(portfolio, modelo, X, dados)
leads.head()

[[0.         1.         1.         1.         1.        ]
 [0.         1.         1.         1.         1.41421356]
 [0.         1.         1.         1.         1.        ]
 ...
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]]
[[   369 101490 215887 308310 124150]
 [   604 218732 366566 308795 317829]
 [  1703 280531 263482 208476   1874]
 ...
 [ 59648 125494  57377 255012  99749]
 [ 93845 461978  42826 117727  94199]
 [226280 240999 106264 119896  21879]]
                                               index  ... idade_media_socios_cat
0  dabe79bec87c88ae04e869bf6bd321ee5e1893cecf6625...  ...          entre 30 e 50
1  1a66d9c4b7fa098a20de6504bcc425145da321cbc8d9a6...  ...          entre 30 e 50
2  472131ba9f5a307f3aec3c06946b0873a9f737dcf63f3d...  ...          entre 30 e 50
3  fd04d93cb5b9c4aef99961587a6138edee28728a0e2af6...  ...          entre 30 e 50
4  ba8b

,index,fl_matriz,de_natureza_juridica,sg_uf,setor,idade_emp_cat,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_veiculo,fl_optante_simples,fl_optante_simei,de_saude_tributaria,de_nivel_atividade,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,contem_filial,socios_cat,idade_media_socios_cat
0,dabe79bec87c88ae04e869bf6bd321ee5e1893cecf6625...,1,EMPRESARIO INDIVIDUAL,PI,COMERCIO,<= 1,0,0,NAO,COMERCIO VAREJISTA,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,0.0,s/info,s/info,CINZA,BAIXA,SUDOESTE PIAUIENSE,SAO RAIMUNDO NONATO,0.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",0,único dono,entre 30 e 50
1,1a66d9c4b7fa098a20de6504bcc425145da321cbc8d9a6...,1,EMPRESARIO INDIVIDUAL,PI,COMERCIO,<= 1,0,1,NAO,COMERCIO VAREJISTA,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,0.0,s/info,s/info,CINZA,BAIXA,SUDOESTE PIAUIENSE,SAO RAIMUNDO NONATO,0.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",0,único dono,entre 30 e 50
2,472131ba9f5a307f3aec3c06946b0873a9f737dcf63f3d...,1,EMPRESARIO INDIVIDUAL,PI,COMERCIO,<= 1,0,1,NAO,COMERCIO VAREJISTA,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,0.0,s/info,s/info,CINZA,BAIXA,SUDOESTE PIAUIENSE,SAO RAIMUNDO NONATO,0.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",0,único dono,entre 30 e 50
3,fd04d93cb5b9c4aef99961587a6138edee28728a0e2af6...,1,EMPRESARIO INDIVIDUAL,PI,COMERCIO,<= 1,0,1,NAO,COMERCIO VAREJISTA,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,0.0,s/info,s/info,CINZA,BAIXA,SUDOESTE PIAUIENSE,SAO RAIMUNDO NONATO,0.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",0,único dono,entre 30 e 50
4,ba8b4694d5da3ec5850872ef42c09a7bed583a1e22762f...,1,EMPRESARIO INDIVIDUAL,PI,COMERCIO,<= 1,0,1,NAO,COMERCIO VAREJISTA,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,0.0,s/info,s/info,CINZA,BAIXA,SUDOESTE PIAUIENSE,SAO RAIMUNDO NONATO,0.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",0,único dono,entre 30 e 50


In [22]:
leads.head()

,index,fl_matriz,de_natureza_juridica,sg_uf,setor,idade_emp_cat,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_veiculo,fl_optante_simples,fl_optante_simei,de_saude_tributaria,de_nivel_atividade,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,contem_filial,socios_cat,idade_media_socios_cat
0,dabe79bec87c88ae04e869bf6bd321ee5e1893cecf6625...,1,EMPRESARIO INDIVIDUAL,PI,COMERCIO,<= 1,0,0,NAO,COMERCIO VAREJISTA,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,0.0,s/info,s/info,CINZA,BAIXA,SUDOESTE PIAUIENSE,SAO RAIMUNDO NONATO,0.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",0,único dono,entre 30 e 50
1,1a66d9c4b7fa098a20de6504bcc425145da321cbc8d9a6...,1,EMPRESARIO INDIVIDUAL,PI,COMERCIO,<= 1,0,1,NAO,COMERCIO VAREJISTA,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,0.0,s/info,s/info,CINZA,BAIXA,SUDOESTE PIAUIENSE,SAO RAIMUNDO NONATO,0.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",0,único dono,entre 30 e 50
2,472131ba9f5a307f3aec3c06946b0873a9f737dcf63f3d...,1,EMPRESARIO INDIVIDUAL,PI,COMERCIO,<= 1,0,1,NAO,COMERCIO VAREJISTA,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,0.0,s/info,s/info,CINZA,BAIXA,SUDOESTE PIAUIENSE,SAO RAIMUNDO NONATO,0.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",0,único dono,entre 30 e 50
3,fd04d93cb5b9c4aef99961587a6138edee28728a0e2af6...,1,EMPRESARIO INDIVIDUAL,PI,COMERCIO,<= 1,0,1,NAO,COMERCIO VAREJISTA,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,0.0,s/info,s/info,CINZA,BAIXA,SUDOESTE PIAUIENSE,SAO RAIMUNDO NONATO,0.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",0,único dono,entre 30 e 50
4,ba8b4694d5da3ec5850872ef42c09a7bed583a1e22762f...,1,EMPRESARIO INDIVIDUAL,PI,COMERCIO,<= 1,0,1,NAO,COMERCIO VAREJISTA,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,0.0,s/info,s/info,CINZA,BAIXA,SUDOESTE PIAUIENSE,SAO RAIMUNDO NONATO,0.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",0,único dono,entre 30 e 50


In [23]:
type(leads)

pandas.core.frame.DataFrame